In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
import pyspark.sql.types as st
from datetime import date

spark = SparkSession.builder.getOrCreate()

daily_country_confirmed_cases = "./data/Daily_Country_Wise_Confirmed_Cases.csv"

daily_df = spark.read.csv(daily_country_confirmed_cases,inferSchema=True,header=True)


In [23]:
# Change to importing this function
def date_check():
    today = date.today()
    return str(today)

from datetime import datetime, timedelta

date_1_week_back = str(date.today() - timedelta(weeks = 1))
date_2_weeks_back = str(date.today() - timedelta(weeks = 2))
date_1_month_back = str(date.today() - timedelta(weeks = 4))

In [29]:
# Showing the change of cases over a month
daily_df.select('Country','2022-05-06', '2022-06-06').where(daily_df['2022-06-06'] >= 1).show()

+-------------+----------+----------+
|      Country|2022-05-06|2022-06-06|
+-------------+----------+----------+
|      England|         1|        73|
|     Portugal|         0|        10|
|        Spain|         0|        17|
|United States|         0|         1|
|        Italy|         0|         6|
|       Canada|         0|         1|
|     Scotland|         0|         2|
|     Slovenia|         0|         3|
|        Wales|         0|         1|
+-------------+----------+----------+



In [26]:
# 
daily_df.select('Country', date_1_week_back).where(daily_df[date_1_week_back] > 10).show()

+-------------+----------+
|      Country|2022-06-28|
+-------------+----------+
|United States|        62|
|      Germany|        65|
|      Belgium|        40|
|        Italy|        32|
|       Canada|        18|
|       France|       109|
|       Norway|        11|
+-------------+----------+



In [ ]:
# daily_df.select('Country', date_1_month_back).where(daily_df[date_1_month_back])